In [1]:
"""
Single-Pass Agentic Flow (beginner-friendly)
Scenario: A user asks about an order. The agent:
  0) reads prior prefs (MEMORY-read)
  1) parses the message (PERCEPTION)
  2) decides the issue type (REASONING)
  3) builds a tiny plan (PLANNING)
  4) executes steps (ACTION)
  5) writes a brief session note (MEMORY-write)

No external services. Tool calls are simple stubs.
"""

# -----------------------------
# (Fake) Data stores and tools
# -----------------------------
# [MEMORY] long-term store (pretend database)
LONG_TERM_MEMORY = {
    "user_42": {"contact": "email", "email": "alex@example.com", "pref_refund": "replacement"},
}

# pretend order DB
ORDER_DB = {
    "A123": {"status": "late", "item": "Headphones", "email": "alex@example.com"},
    "B456": {"status": "shipped", "item": "Keyboard", "email": "alex@example.com"},
}

# --- "Tool" stubs (these are ACTIONs when called) ---
def tool_lookup_order(order_id):
    # [ACTION] external read (stub)
    return ORDER_DB.get(order_id)

def tool_create_replacement(order_id):
    # [ACTION] external write (stub)
    return {"new_order_id": f"{order_id}-R", "status": "created"}

def tool_send_email(to, subject, body):
    # [ACTION] side effect (stub: print instead of real email)
    print(f"\n[ACTION] Email to {to}\nSubject: {subject}\n{body}")

# ---------------------------------
# 0) MEMORY (read): load user prefs
# ---------------------------------
def load_user_profile(user_id):
    # [MEMORY - READ] fetch prior preferences if any
    return LONG_TERM_MEMORY.get(user_id, {})

# ---------------------------------
# 1) PERCEPTION: parse the message
# ---------------------------------
def parse_user_message(msg: str):
    """
    Very simple parser:
      - find an order id like A123/B456 (toy regex-free)
      - detect keywords for issue hints
    """
    # [PERCEPTION] turn raw text into structured fields
    tokens = msg.replace(",", " ").replace(".", " ").split()
    order_id = None
    for t in tokens:
        if t.upper().startswith(("A", "B")) and t[1:].isdigit():
            order_id = t.upper()
            break
    hints = {
        "late": any(w in msg.lower() for w in ["late", "delay", "yesterday", "not arrived"]),
        "damaged": any(w in msg.lower() for w in ["broken", "damaged", "cracked"]),
        "lost": any(w in msg.lower() for w in ["lost", "missing"]),
    }
    return {"order_id": order_id, "hints": hints, "original": msg}

# ---------------------------------
# 2) REASONING: decide the issue
# ---------------------------------
def classify_issue(structured, order_record):
    """
    Tiny rule-based "reasoner":
      - use hints + order status to decide: LATE / DAMAGED / LOST / UNKNOWN
    """
    # [REASONING]
    if order_record is None:
        return "UNKNOWN"

    status = order_record["status"]
    h = structured["hints"]

    if h["damaged"]:
        return "DAMAGED"
    if h["lost"]:
        return "LOST"
    if h["late"] or status in ("late", "delayed"):
        return "LATE"
    if status == "shipped":
        return "IN_TRANSIT"
    return "UNKNOWN"

# ---------------------------------
# 3) PLANNING: build the steps
# ---------------------------------
def plan_resolution(issue_type, order_id, user_profile):
    """
    Return an ordered list of steps (strings) the agent will execute.
    Keep it small and readable.
    """
    # [PLANNING]
    steps = []
    if issue_type == "LATE":
        steps = [
            f"confirm_contact({user_profile.get('contact','email')})",
            f"create_replacement({order_id})",
            "notify_user(replacement_created)",
        ]
    elif issue_type == "DAMAGED":
        steps = [
            "request_photos()",
            f"create_replacement({order_id})",
            "notify_user(replacement_created)",
        ]
    elif issue_type == "IN_TRANSIT":
        steps = [
            "share_tracking()",
            "set_expectation_window()",
        ]
    else:
        steps = [
            "ask_for_details()",
        ]
    return steps

# ---------------------------------
# 4) ACTION: execute the plan
# ---------------------------------
def execute_steps(steps, order_record, user_profile):
    """
    Walk the steps once (no multi-turn loop). Each branch calls a stub tool.
    """
    # [ACTION]
    result = {"message": "", "replacement": None}
    contact_email = user_profile.get("email") or (order_record["email"] if order_record else None)

    for step in steps:
        if step.startswith("confirm_contact"):
            # no-op in this demo
            pass

        elif step.startswith("create_replacement"):
            rep = tool_create_replacement(order_record["item"] if order_record else "N/A")
            result["replacement"] = rep

        elif step == "notify_user(replacement_created)":
            if contact_email:
                tool_send_email(
                    contact_email,
                    "Replacement order created",
                    "We created a replacement and will send tracking shortly."
                )

        elif step == "request_photos()":
            if contact_email:
                tool_send_email(
                    contact_email,
                    "A quick check to help you faster",
                    "Please reply with photos of the damage so we can process a replacement."
                )

        elif step == "share_tracking()":
            if contact_email:
                tool_send_email(
                    contact_email,
                    "Your order is on the way",
                    "Your package is in transit. Estimated arrival: 2–3 days."
                )

        elif step == "set_expectation_window()":
            # no-op: would set an SLA/reminder in a real system
            pass

        elif step == "ask_for_details()":
            print("\n[ACTION] Ask user: Could you share your order ID and what's wrong?")

    # Build a single human-friendly message for the console
    if result["replacement"]:
        result["message"] = "Replacement initiated. You’ll receive an email with details."
    return result

# ---------------------------------
# 5) MEMORY (write): session note
# ---------------------------------
def write_session_summary(user_id, order_id, issue_type, action_result):
    # [MEMORY - WRITE] store a tiny summary (in a real app: DB/file)
    LONG_TERM_MEMORY[user_id] = {
        **LONG_TERM_MEMORY.get(user_id, {}),
        "last_issue": issue_type,
        "last_order": order_id,
        "last_action": action_result.get("message", "n/a"),
    }

# ===========================
# Run the single-pass pipeline
# ===========================
if __name__ == "__main__":
    # Input: one message, one user
    user_id = "user_42"
    user_msg = "Hi, where is my order A123? It was supposed to arrive yesterday."

    # 0) MEMORY-read
    profile = load_user_profile(user_id)
    print("[MEMORY-read]", profile)

    # 1) PERCEPTION
    structured = parse_user_message(user_msg)
    print("[PERCEPTION]", structured)

    # 2) REASONING
    order = tool_lookup_order(structured["order_id"])
    issue = classify_issue(structured, order)
    print("[REASONING] issue_type =", issue)

    # 3) PLANNING
    steps = plan_resolution(issue, structured["order_id"], profile)
    print("[PLANNING] steps =", steps)

    # 4) ACTION
    result = execute_steps(steps, order, profile)
    print("[ACTION] result =", result)

    # 5) MEMORY-write
    write_session_summary(user_id, structured["order_id"], issue, result)
    print("[MEMORY-write] new snapshot:", LONG_TERM_MEMORY[user_id])


[MEMORY-read] {'contact': 'email', 'email': 'alex@example.com', 'pref_refund': 'replacement'}
[PERCEPTION] {'order_id': None, 'hints': {'late': True, 'damaged': False, 'lost': False}, 'original': 'Hi, where is my order A123? It was supposed to arrive yesterday.'}
[REASONING] issue_type = UNKNOWN
[PLANNING] steps = ['ask_for_details()']

[ACTION] Ask user: Could you share your order ID and what's wrong?
[ACTION] result = {'message': '', 'replacement': None}
[MEMORY-write] new snapshot: {'contact': 'email', 'email': 'alex@example.com', 'pref_refund': 'replacement', 'last_issue': 'UNKNOWN', 'last_order': None, 'last_action': ''}
